In [1]:
from gurobi import *

In [2]:
rack_name, slots = multidict({
    'alpha' : 48
})

In [3]:
chips, quantity, powerdraw, outlets_required, slots_required = multidict({
    '1080' : [5, 45, 2, 1],
    '2080' : [5, 55, 2, 2]
})

In [4]:
def solve(rack_name, slots,chips, quantity, powerdraw, slots_required):

    m = Model()

    # always need the load on each phase Lead
    L = m.addVars(3, name='phases')

    # add x_ij where i is the chip and j is the possible orientations
    # assume only 2 slots necessary
    # orientation
    # 1 = 1,2
    # 2 = 1,3
    # 3 = 2,3
    # again the set is not an SOS
    indicies = [(chip,orientation) for chip in chips for orientation in range(3)] # gen pairs 
    x = m.addVars(indicies,vtype=GRB.INTEGER)

    # calculate the power across phases
    m.addConstr(quicksum(powerdraw[chip]*(x[chip,0]+x[chip,1]) for chip in chips),GRB.EQUAL,L[0]);
    m.addConstr(quicksum(powerdraw[chip]*(x[chip,0]+x[chip,2]) for chip in chips),GRB.EQUAL,L[1]);
    m.addConstr(quicksum(powerdraw[chip]*(x[chip,1]+x[chip,2]) for chip in chips),GRB.EQUAL,L[2]);
    
    # slot locations
    m.addConstr(quicksum(x[chip, 0] for chip in chips) <= 7)
    m.addConstr(quicksum(x[chip, 1] for chip in chips) <= 2)
    m.addConstr(quicksum(x[chip, 2] for chip in chips) <= 7)
    

    m.addConstrs(quicksum(x[chip,orient] for orient in range(3)) == quantity[chip] for chip in chips);

    # simplifies later constraints
    m.addConstr(L[0],GRB.LESS_EQUAL,L[1])
    m.addConstr(L[1],GRB.LESS_EQUAL,L[2])

    m.setObjective(L[2]-L[0])

    m.optimize()
    
    return m.getAttr('x',x)

In [23]:
solve(rack_name, slots,chips, quantity, powerdraw, slots_required)

Academic license - for non-commercial use only
Optimize a model with 10 rows, 9 columns and 31 nonzeros
Variable types: 3 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 7e+00]
Presolve removed 4 rows and 3 columns
Presolve time: 0.00s
Presolved: 6 rows, 6 columns, 21 nonzeros
Variable types: 0 continuous, 6 integer (0 binary)
Found heuristic solution: objective 170.0000000

Root relaxation: cutoff, 3 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0       170.00000  170.00000  0.00%     -    0s

Explored 0 nodes (3 simplex iterations) in 0.02 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 170 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.700000000000e+0

{('1080', 0): -0.0,
 ('1080', 1): -0.0,
 ('1080', 2): 5.0,
 ('2080', 0): 2.0,
 ('2080', 1): 2.0,
 ('2080', 2): 1.0}

In [5]:
import pandas as pd

In [21]:
index = pd.MultiIndex.from_tuples([(rack,i) for rack in rack_name for i in range(48)],names = ['rack','slot'])

In [22]:
pd.DataFrame(index=index)

Empty DataFrame
Columns: []
Index: [(alpha, 0), (alpha, 1), (alpha, 2), (alpha, 3), (alpha, 4), (alpha, 5), (alpha, 6), (alpha, 7), (alpha, 8), (alpha, 9), (alpha, 10), (alpha, 11), (alpha, 12), (alpha, 13), (alpha, 14), (alpha, 15), (alpha, 16), (alpha, 17), (alpha, 18), (alpha, 19), (alpha, 20), (alpha, 21), (alpha, 22), (alpha, 23), (alpha, 24), (alpha, 25), (alpha, 26), (alpha, 27), (alpha, 28), (alpha, 29), (alpha, 30), (alpha, 31), (alpha, 32), (alpha, 33), (alpha, 34), (alpha, 35), (alpha, 36), (alpha, 37), (alpha, 38), (alpha, 39), (alpha, 40), (alpha, 41), (alpha, 42), (alpha, 43), (alpha, 44), (alpha, 45), (alpha, 46), (alpha, 47)]